In [2]:
library(ArchR)
library(Matrix)
library(dplyr)
library(GenomicRanges)
library(Biostrings)
source("/ailab/user/fanyimin/CellSpace-master/tutorial/variable-peaks/IterativeLSI.R")
library(GenomicRanges)
library(Matrix)
library(data.table)
library(dplyr)
library(BSgenome.Hsapiens.UCSC.hg38)



                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [3]:
peaks <- fread(
  file = "var.csv",
  header = TRUE,      
  stringsAsFactors = FALSE
)

peaks.gr <- makeGRangesFromDataFrame(
  df = peaks,
  seqinfo = seqinfo(BSgenome.Hsapiens.UCSC.hg38)
)


In [ ]:
cellname <- fread(
  file = "obs.csv",
  header = TRUE,     
  stringsAsFactors = FALSE
)
cellname$label
counts <- readMM("matrix.mtx")

In [ ]:
counts@x[counts@x != 0] <- 1
dimnames(counts) <- list(cellname$label, as.character(peaks.gr))

In [ ]:
genome <- BSgenome.Hsapiens.UCSC.hg38
getSeq(peaks.gr, x = genome) %>%
  writeXStringSet(filepath = "var_peaks.fa")
write(rownames(counts), ncolumns = 1, file = "cell-names.txt")
writeMM(counts, file = "cell_by_peak-counts.mtx")


#CellSpace \
  #-output data/CellSpace_embedding-var_tiles \
  #-cpMat  data/cell_by_tile-counts.mtx \
  #-peaks  data/var_tiles.fa


In [2]:
library(CellSpace)
library(TFBSTools)
library(JASPAR2020)
library(dplyr)
source("/ailab/user/fanyimin/CellSpace-master/tutorial/plot-functions.R")

In [3]:
cell.idx <- readLines("cell-names.txt")
cso <- CellSpace(
  project = "ATAC",
  emb.file = "CellSpace_embedding-var_tiles.tsv",
  meta.data = as.data.frame(cell.idx)
)

In [4]:
cso <- find_neighbors(cso, n.neighbors = 20) %>%
  find_clusters(resolution = c(0.8, 1.2)) %>%
  run_UMAP(n.neighbors = 20, min.dist = 0.2, spread = 1)

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 8981
Number of edges: 333309

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8492
Number of communities: 13
Elapsed time: 0 seconds
Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 8981
Number of edges: 333309

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8050
Number of communities: 14
Elapsed time: 0 seconds


Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
23:38:16 UMAP embedding parameters a = 1.262 b = 1.003

23:38:16 Read 8981 rows and found 30 numeric columns

23:38:16 Using Annoy for neighbor search, n_neighbors = 20

23:38:16 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

23:38:17 Writing NN index file to temp file /tmp/RtmpGSdPwp/file386460e34d7d

23:38:17 Searching Annoy index using 1 thread, search_k = 2000

23:38:19 Annoy recall = 100%

23:38:20 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 

In [ ]:
library(RColorBrewer)

n <- length(unique(cso@meta.data$cell.idx))
print(n)
pal <- colorRampPalette(brewer.pal(12, "Paired"))(14)

plot.groups(
  cso, vis = "cells_UMAP", groups = "cell.idx",
  pal = pal, add.labels = T
)

In [ ]:

pwm_txt <- readLines("motif.txt")

parse_pwm <- function(txt_lines, selected_motifs) {
  pwm_list <- list() 
  i <- 1
  while (i <= length(txt_lines)) {
    if (startsWith(txt_lines[i], ">")) {
      full_name <- sub("^>", "", txt_lines[i]) 
      parts <- strsplit(full_name, "\t")[[1]]  
      motif_name <- parts[2] 
      
      if (motif_name %in% selected_motifs) {
        i <- i + 1
        pwm_matrix <- NULL
        bases <- c("A", "C", "G", "T")
        for (j in 1:4) {
          base_line <- txt_lines[i]
          base_values <- as.numeric(unlist(strsplit(gsub("[^0-9 ]", "", base_line), "\\s+")))
          
          if (is.null(pwm_matrix)) {
            pwm_matrix <- matrix(NA, nrow = 4, ncol = length(base_values))
          }
          
          if (length(base_values) == ncol(pwm_matrix)) {
            pwm_matrix[j, ] <- base_values
          } else {
            stop(paste("Wrong Number!"))
          }
          
          i <- i + 1
        }
        
        rownames(pwm_matrix) <- bases
        pwm_matrix <- pwm_matrix[, -1]  
      
        pwm_list[[motif_name]] <- pwm_matrix
      } else {
        i <- i + 5
      }
    }
  }
  return(pwm_list)
}

selected_motifs <- c('PAX6', 'INSM1', 'SOX9', 'EMX2', 'LHX2', 'FOXG1', 
                     'TCF4', 'TCF3', 'TFAP2C', 'FOS', 'POU3F3', 'NEUROD2', 
                     'BHLHE22', 'JUND', 'MEF2C', 'POU2F2', 'NFIA', 'MEIS2', 'EOMES')

pwm_list <- parse_pwm(pwm_txt, selected_motifs)

#print(pwm_list)


In [ ]:
pwm_to_pwmat <- function(pwm_list) {
  pwmat_list <- list()
  
  for (motif_name in names(pwm_list)) {
    pwm_matrix <- pwm_list[[motif_name]]
    #print(pwm_matrix)
    pwmat <- PWMatrix(
      ID = motif_name,
      name = motif_name,
      matrixClass = "Unknown",
      strand = "*",
      bg = c(A = 0.25, C = 0.25, G = 0.25, T = 0.25),  
      profileMatrix = pwm_matrix, 
      pseudocounts = 1e-4,
      tags = list()
    )
      
    pwmat_list[[motif_name]] <- pwmat
  }

  return(pwmat_list)
}

pwmat_list <- pwm_to_pwmat(pwm_list)

#print(pwmat_list)


In [18]:
motif.emb <- lapply(pwmat_list, function(pwm){
  motif_embedding(cso, PWM = pwm)
}) %>% do.call(what = rbind)

In [19]:
motif.score <- cosine_similarity(x = cso@cell.emb, y = motif.emb) %>% scale()

In [ ]:
md <- cso@meta.data[, c("cell.idx", "Clusters.res_0.8", "Clusters.res_1.2")]

clusters_0.8 <- unique(md$Clusters.res_0.8)
clusters_1.2 <- unique(md$Clusters.res_1.2)
cell.idx_uq <- unique(md$cell.idx)

pal <- list(
  cell.idx = setNames(rainbow(length(cell.idx_uq)), cell.idx_uq),
  Clusters.res_0.8 = setNames(rainbow(length(clusters_0.8)), clusters_0.8),
  Clusters.res_1.2 = setNames(rainbow(length(clusters_1.2)), clusters_1.2)
)

#print(pal)
plot.scores(t(motif.score), cell.annot = md, pal = pal, column_split = cso$Cell_type)

In [26]:

write.csv(motif.score, file = "motif_score.csv", row.names = TRUE)

write.csv(as.data.frame(cso@cell.emb), file = "cell_embedding.csv", row.names = TRUE)

write.csv(as.data.frame(cso@kmer.emb), file = "motif_embedding.csv", row.names = TRUE)
